# Exercise 9: Decorrelation

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F

import scipy.stats as stats

import pickle

Below we create a simple dataset. The signal consists of 5 uncorrelated Gaussian distributions, the background is flat

In [ ]:
sig = np.random.normal(size=(2000,5))
bg = np.random.uniform(low=-3, high=3, size=(2000,5))

data = np.concatenate([sig,bg]) # combine signal and background
labels = np.concatenate([np.ones(2000), np.zeros(2000)]) # and also create labels - one for signal and zero for background

# shuffle the examples and indices
idx = np.arange(data.shape[0])
np.random.shuffle(idx)
data = data[idx]
labels = labels[idx]

# select subsets as training and testing data
X_train = data[:1500]
y_train = labels[:1500]

X_test = data[1500:]
y_test = labels[1500:]

# and visualize the distributions
def plot2d(xdim, ydim):
    plt.clf()
    plt.scatter(sig[:,xdim],sig[:,ydim],color='blue',alpha=0.5)
    plt.scatter(bg[:,ydim],bg[:,xdim],color='orange',alpha=0.5)
    plt.xlabel("Dim {0}".format(xdim))
    plt.ylabel("Dim {0}".format(ydim))
    plt.show()
    
plot2d(0,1)
plot2d(0,2)
plot2d(0,3)
plot2d(0,4)
plot2d(1,2)
plot2d(1,3)
plot2d(1,4)
plot2d(2,3)
plot2d(2,4)
plot2d(3,4)


**Define neural network and training hyperparameters.**

In [ ]:
class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet, self).__init__()
        
        # DEFINE THE NETWORK LAYERS
        # 1st LAYER should have the same size as inputs, last layer=1

    def forward(self, x):
        

        # IMPLEMENT FORWARD PASS 
        x = F.sigmoid(x)

        return x
    
# Which device to use for NN calculations
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  

# Create network object
model = NeuralNet().to(device)

# Adam optimiser
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# how many epochs to train for
n_epochs = 20 

# how many examples / batch
batch_size = 5

# Keep track of the accuracies
train_accs = []
test_accs = []

# Training loop and evaluation below


In [ ]:
train_examples = X_train.shape[0]
n_batches = int(train_examples/batch_size)

# Loop over the epochs
for ep in range(n_epochs):
    
    # Each epoch is a complete loop over the training data
    for i in range(n_batches):
        
        # Reset gradient
        optimizer.zero_grad()
        
        i_start = i*batch_size
        i_stop  = (i+1)*batch_size
        
        # Convert x and y to proper objects for PyTorch
        x = torch.tensor(X_train[i_start:i_stop],dtype=torch.float)
        y = torch.tensor(y_train[i_start:i_stop],dtype=torch.long)

        
        # Apply the network 
        net_out = model(x)
        
        loss = F.binary_cross_entropy(net_out,y.float())
        
    
        # Calculate the gradients
        loss.backward()

        # Update the weights
        optimizer.step()
        

    # end of loop over batches
        
    # Calculate predictions on training and testing data
    y_pred_train = model(torch.tensor(X_train,dtype=torch.float)).detach().numpy()[:,0]
    y_pred_test  = model(torch.tensor(X_test,dtype=torch.float)).detach().numpy()[:,0]
    
    # Calculate accuracy on training and testing data
    train_acc = sum(y_train.astype(bool) == (y_pred_train>0.5)) / y_train.shape[0]
    test_acc = sum(y_test.astype(bool) == (y_pred_test>0.5)) / y_test.shape[0]
    
    # print some information
    print("Epoch:",ep, "Train Accuracy:", train_acc,  "Test Accuracy:", test_acc)
    
    # and store the accuracy for later use
    train_accs.append(train_acc)
    test_accs.append(test_acc)
# end of loop over epochs
    
# Prepare and show accuracy over time
plt.axis('on')
plt.plot(train_accs,label="train")
plt.plot(test_accs,label="test")
plt.legend()
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.show()

print("Best test accuracy:",max(test_accs))


In [ ]:
# Helper function to calculate the jensen shannon divergence
def calc_js(P,Q,nbins=25):

    # find lowest and highest value
    low = min(min(P), min(Q))
    high = max(max(P), max(Q))

    # turn distributions into histograms
    P_h,_ = np.histogram(P, bins=nbins, range=(low,high),density=True)
    Q_h,_ = np.histogram(Q, bins=nbins, range=(low,high),density=True)

    # calculate average histgrogram
    M_h = 0.5 * (P_h + Q_h)
    
    # and add KL divergences
    JS = 0.5 * stats.entropy(P_h,M_h) + 0.5 * stats.entropy(Q_h,M_h)
    
    return JS

# Evaluate DNN on testing data
x = torch.tensor(X_test,dtype=torch.float)
y_pred = model(x).detach().numpy()[:,0]

# Find the threshold corresponding to 70% efficiency for true signal events
threshold = np.percentile(y_pred[y_test==1],70)

# select events that pass (ie lie above that threshold)
passed_selection = (y_pred > threshold)

# Plot the distribution of all events before and after the selection
_ = plt.hist(X_test[:,0],bins=20,density=True,range=(0,4),alpha=0.5)
_ = plt.hist(X_test[passed_selection,0],bins=20,density=True,range=(0,4),alpha=0.5)

# and calculate the jensen shannon divergence
calc_js(X_test[:,0],X_test[passed_selection,0])

# Homework


  * Implement a fully connected network using all 5 input variables. What is the best accuracy it obtains and how strong does it affect the distribution of variable 0 (the code to draw the distribution and calculate the JS divergence are provided above)?
  
  * By how much does the accuracy and shaping change if you do NOT use the first input variable. You will need to change the network and all instances where the data is passed to the model. You can use `X[:,1:]` to select the sub-tensor without the first column.
  
  * Calculate weights that flatten the distribution of the first input variable (but use it as input to the network). Helpful numpy functions are `np.histogram` and `np.digitize`. The `binary_cross_entropy` loss functions has an argument `weight` that accepts a tensor with the same shape as `y` that contains the weight. Calculate the weight for each example and use in training. How much does this change accuracy and shaping?